<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">LLM Ops - Tracing, Evaluation, and Analysis</h1>

In this tutorial we will learn how to build, observe, evaluate, and analyze a LLM powered application. 

It has the following sections:

1. Understanding LLM-powered applications
2. Observing the application using traces
3. Evaluating the application using LLM Evals
4. Exploring and and troubleshooting the application using UMAP projection and clustering


## Understanding LLM powered applications

Building software with LLMs, or any machine learning model, is [fundamentally different](https://karpathy.medium.com/software-2-0-a64152b37c35). Rather than compiling source code into binary to run a series of commands, we need to navigate datasets, embeddings, prompts, and parameter weights to generate consistent accurate results. LLM outputs are probabilistic and therefore don't produce the same deterministic outcome every time.

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/5_steps_of_building_llm_app.png" width="1100" />

There's a lot that can go into building an LLM application, but let's focus on the architecture. Below is a diagram of one possible architecture. In this diagram we see that the application is built around an LLM but that there are many other components that are needed to make the application work.

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/llm_app_architecture.png" width="1100" />

The complexity that is involved in building an LLM application is why observability is so important. Observability is the ability to understand the internal state of a system by examining its inputs and outputs. Each step of the response generation process needs to be monitored, evaluated and tuned to provide the best possible experience. Not only that, certain tradeoffs might need to be made to optimize for speed, cost, or accuracy. In the context of LLM applications, we need to be able to understand the internal state of the system by examining telemetry data such as LLM Traces.


## Observing the application using traces

LLM Traces and Observability lets us understand the system from the outside, by letting us ask questions about that system without knowing its inner workings. Furthermore, it allows us to easily troubleshoot and handle novel problems (i.e. “unknown unknowns”), and helps us answer the question, “Why is this happening?”

LLM Traces are designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context such as retrieval from vector stores and the usage of external tools such as search engines or APIs. It lets you understand the inner workings of the individual steps your application takes wile also giving you visibility into how your system is running and performing as a whole.

Traces are made up of a sequence of `spans`. A span represents a unit of work or operation (think a span of time). It tracks specific operations that a request makes, painting a picture of what happened during the time in which that operation was executed.

LLM Tracing supports the following span kinds:

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/span_kinds.png" width="1100"/>


By capturing the building blocks of your application while it's running, phoenix can provide a more complete picture of the inner workings of your application. To illustrate this, let's look at an example LLM application and inspect it's traces.

### Traces and Spans in action

Let's build a relatively simple LLM-powered application that will answer questions about Arize AI. This example uses LlamaIndex for RAG and OpenAI as the LLM but you could use any LLM you would like. The details of the application are not important, but the architecture is. Let's get started.

In [ ]:
%pip install -qq arize-phoenix==7.3.2 llama-index-core==0.12.8 llama-index-llms-ollama==0.5.0 llama-index-embeddings-ollama==0.5.0 llama-index-readers-file==0.4.1 gcsfs==2024.12.0 nest_asyncio==1.6.0 openinference-instrumentation-llama_index==3.1.2 litellm==1.55.12

In [ ]:
import phoenix as px

session = px.launch_app()

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

You'll need to make sure you have ollama installed and running both models `ollama/mistral:7b` and `ollama/nomic-embed-text`.

To do this, run the following commands in your terminal:

```bash
ollama pull mistral:7b
ollama pull nomic-embed-text
```

Then we'll need to set the `OLLAMA_API_BASE` environment variable to point to your local ollama instance. The port `11434` is the default port for ollama.

In [5]:
ollama_model = "mistral:7b"
ollama_embed_model = "nomic-embed-text"

In [6]:
import os

os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"

In [7]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
)
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# download and install dependencies for benchmark dataset
rag_dataset, documents = download_llama_dataset("PaulGrahamEssayDataset", "./data")

Settings.llm = Ollama(model=ollama_model)
Settings.embed_model = OllamaEmbedding(model_name=ollama_embed_model)

# build basic RAG system
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What did the Paul Graham do at Y Combinator?")
print(response)

Let's test a few more queries to see if the application is working as expected.

In [ ]:
from tqdm import tqdm

queries = [
    "What was Paul Graham's role at Y Combinator?",
    "What are Paul Graham's views on startups?",
    "What does Paul Graham think about programming languages?",
    "How did Paul Graham become successful as an entrepreneur?",
]

for query in tqdm(queries):
    response = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")

Now that we've run the application a few times, let's take a look at the traces in the UI

In [ ]:
print("The Phoenix UI:", session.url)

The UI will give you an interactive troubleshooting experience. You can sort, filter, and search for traces. You can also view the detail of each trace to better understand what happened during the response generation process.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/trace_details_view.png" width="1100"/>

In addition to being able to view the traces in the UI, you can also query for traces to use as pandas dataframes in your notebook.

In [ ]:
spans_df = px.Client().get_spans_dataframe()
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

With just a few lines of code, we have managed to gain visibility into the inner workings of our application. We can now better understand how things like retrieval, prompts, and parameter weights could be affecting our application. But what can we do with this information? Let's take a look at how to use LLM evals to evaluate our application.

In [12]:
if (
    input("The tutorial is about to move on to the evaluation section. Continue [Y/n]?")
    .lower()
    .startswith("n")
):
    assert False, "notebook stopped"

## Evaluating the application using LLM Evals

Evaluation should serve as the primary metric for assessing your application. It determines whether the app will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.


Let's first convert our traces into a workable datasets

In [13]:
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
queries_df = get_qa_with_reference(px.Client())

We can now use Phoenix's LLM Evals to evaluate these queries. LLM Evals uses an LLM to grade your application based on different criteria. For this example we will use the evals library to see if any `hallucinations` are present and if the `Q&A Correctness` is good (whether or not the application answers the question correctly).

In [ ]:
import nest_asyncio

from phoenix.evals import (
    HALLUCINATION_PROMPT_RAILS_MAP,
    HALLUCINATION_PROMPT_TEMPLATE,
    QA_PROMPT_RAILS_MAP,
    QA_PROMPT_TEMPLATE,
    LiteLLMModel,
    llm_classify,
)

nest_asyncio.apply()

# Check if the application has any indications of hallucinations
hallucination_eval = llm_classify(
    dataframe=queries_df,
    model=LiteLLMModel(model="ollama/" + ollama_model),
    template=HALLUCINATION_PROMPT_TEMPLATE,
    rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,  # Makes the LLM explain its reasoning
)
hallucination_eval["score"] = (
    hallucination_eval.label[~hallucination_eval.label.isna()] == "factual"
).astype(int)

# Check if the application is answering questions correctly
qa_correctness_eval = llm_classify(
    dataframe=queries_df,
    model=LiteLLMModel(model="ollama/" + ollama_model),
    template=QA_PROMPT_TEMPLATE,
    rails=list(QA_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,  # Makes the LLM explain its reasoning
)

qa_correctness_eval["score"] = (
    qa_correctness_eval.label[~qa_correctness_eval.label.isna()] == "correct"
).astype(int)

In [ ]:
hallucination_eval.head()

In [ ]:
qa_correctness_eval.head()

As you can see from the results, one of the queries was flagged as a hallucination. Let's log these results to the phoenix server to view the hallucinations in the UI.

In [17]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval),
)

Now that we have the hallucinations logged, let's take a look at them in the UI. You will notice that the traces that correspond to hallucinations are clearly marked in the UI and you can now query for them!

In [ ]:
print("The Phoenix UI:", session.url)

We've now identified that there are certain queries that are resulting in hallucinations or incorrect answers. Let's see if we can use LLM Evals to identify if these issues are caused by the retrieval process for RAG. We are going to use an LLM to grade whether or not the chunks retrieved are relevant to the query.

In [ ]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    llm_classify,
)

retrieved_documents_eval = llm_classify(
    dataframe=retrieved_documents_df,
    model=LiteLLMModel(model="ollama/" + ollama_model),
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,
)

retrieved_documents_eval["score"] = (
    retrieved_documents_eval.label[~retrieved_documents_eval.label.isna()] == "relevant"
).astype(int)

In [ ]:
retrieved_documents_eval.head()

Looks like we are getting a lot of irrelevant chunks of text that might be polluting the prompt sent to the LLM. Let's log these evals to phoenix, at which point phoenix will automatically calculate retrieval metrics for us.

In [21]:
from phoenix.trace import DocumentEvaluations

px.Client().log_evaluations(
    DocumentEvaluations(eval_name="Relevance", dataframe=retrieved_documents_eval)
)

If we once again visit the UI, we will now see that Phoenix has aggregated up retrieval metrics (`precision`, `ndcg`, and `hit`). We see that our hallucinations and incorrect answers directly correlate to bad retrieval!

In [ ]:
print("The Phoenix UI:", session.url)

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/screenshots/document_evals_on_traces.png" />

There are many more evaluations metrics that can be used to make determinations about the quality of your application. Phoenix supports evaluating not only traces but individual spans (such as retrievers) as well as performing retrieval analysis for your document chunks. Evaluation metrics can also be customized for your specific use-case. For more details, consult the phoenix docs.

## Conclusion

Phoenix is a powerful companion for building LLM-powered applications. It allows you to observe, evaluate, and explore your application at every step of you development process. For more details, consult the [phoenix docs](https://docs.arize.com/phoenix).